# Spatial Resolution GIF Maker
stough 202-

Making an animation similar to the interactive slider in the [spatial resolution demo](./spatial_resolution.ipynb). 

Some pointers:
- [neat, tight example](http://louistiao.me/posts/notebooks/save-matplotlib-animations-as-gifs/)
- [even simpler](https://riptutorial.com/matplotlib/example/23562/save-animation-to-gif)

Had some issues trying to pad the figure correctly so that that animation showed the changing axis titles without putting too much padding on the other sides. Turns out setting the bottom of the `rect` argument to a slightly negative value allows the axis titles to be visible. Another issue was that the first frame of the saved gif had transparent background. As per [here](https://stackoverflow.com/questions/59715466/how-to-control-matplotlib-figure-patch-facecolor-changes-when-animating) and [here](https://adrian.pw/blog/matplotlib-transparent-animation/), I saw to add the `savefig_kwargs` dictionary to the gif saving code, forcing transparency off and sticking to white background. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import rescale

from matplotlib import animation, rc
from IPython.display import HTML, Image

In [ ]:
rc('animation', html='html5')

In [ ]:
# Ripped from the spatial_resolution demo
def downandup(I, factor):
    # downscale
    sI = rescale(I, 1/factor,
                 order=0, # use nearest neighbor
                 anti_aliasing=False,
                 multichannel=True)

    # rescale back up
    reI = rescale(sI, factor,
                  order=0, # use nearest neighbor sampling
                  anti_aliasing=False,
                  multichannel=True)
    return reI

In [ ]:
# Load the image
I = plt.imread('../dip_pics/grandCanyon.jpg')
Ire = downandup(I, factor=1)

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots(1,2, figsize=(10,3))
[a.axes.get_xaxis().set_visible(False) for a in ax];
[a.axes.get_yaxis().set_visible(False) for a in ax];

plt.tight_layout(rect=(0,-.1, 1,1))

ax[0].imshow(I, interpolation=None)
ax[0].set_title('Original')

# ax[1].imshow(Ire, interpolation=None)
# ax[1].set_title('Quarter Res')
rdisp = ax[1].imshow(I, interpolation=None)
rtext = ax[1].set_title(f'Factor: {1:d}')

In [ ]:
def init():
    rdisp.set_data(Ire)
    rtext.set_text(f'Factor: {1:d}')
    return (rdisp, rtext,)

In [ ]:
# animation function. This is called sequentially
def animate(i):
    Ire = downandup(I, factor=2**i)
    rdisp.set_data(Ire)
    rtext.set_text(f'Factor: {2**i:d}')
    return (rdisp, rtext,)

In [ ]:
# call the animator. blit=True means only re-draw the parts that 
# have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=9, interval=350, blit=True)

In [ ]:
anim

In [ ]:
anim.save('../dip_outs/spatial_res_animation.gif', writer='imagemagick', fps=2,
          savefig_kwargs={'transparent': False, 'facecolor': 'white'})

In [ ]:
Image(url='../dip_outs/spatial_res_animation.gif')